In [ ]:
!pip install pandas==1.1.5 numpy==1.19.4 scipy==1.5.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install websockets>=9.0


In [ ]:
!pip install alpaca-trade-api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import alpaca_trade_api as tradeapi
import pandas as pd
import yfinance as yf
# API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKZBF19BMWCUWI0SA9E3"
ALPACA_SECRET_KEY = "uX2AdG5eqYzu80wf5PsjbrbvFoKR9NOx2iuDQHfP"

# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, 
                    base_url=BASE_URL, api_version='v2')
import logging
import threading
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
import datetime
import yfinance as yf

import nest_asyncio
nest_asyncio.apply()

In [ ]:
#HEAVY WORK IN PROGRESS

In [ ]:
#This is a paper trade arbitrage bot that executes a simple strategy based on the relative returns of a leveraged ticker (e.g. UPRO) vs. an unleveraged equivalent (e.g. SPY)

#blagh = divergence from theoretical value

In [ ]:
# Utility to truncate a float value to a certain number of decimal places.
# We'll use this to see if a "penny level" was crossed when we compare prices.
# This is necessary because a price can change by 1/100th of a penny, but we
# can only trade at full-penny increments.
def truncate(val, decimal_places):
    return int(val * 10**decimal_places) / 10**decimal_places


# The MartingaleTrader bets that streaks of increases or decreases in a stock's
# price are likely to break, and increases its bet each time it is wrong.
class LEFTTRADER(object):
    def __init__(self):
        # API authentication keys can be taken from the Alpaca dashboard.
        # https://app.alpaca.markets/paper/dashboard/overview
        self.key_id = 'PKZBF19BMWCUWI0SA9E3'
        self.secret_key = 'uX2AdG5eqYzu80wf5PsjbrbvFoKR9NOx2iuDQHfP'
        self.base_url = "https://paper-api.alpaca.markets"
        self.data_url = "https://paper-api.alpaca.markets"

        # The symbol we will be trading
        self.tick_symbol = 'SPY'
        self.levtick_symbol = 'UPRO'

        #the opening price of each symbol
        SPYTICK = yf.Ticker("SPY")
        UPROTICK = yf.Ticker("UPRO")
        self.tick_open = round(SPYTICK.history(period="1d")['Open'].values[0], 2)
        self.levtick_open = round(UPROTICK.history(period="1d")['Open'].values[0], 2)

        #the last price of each symbol
        self.last_tick_val = 0
        self.last_levtick_val = 0
        

        #the running mean of each symbol, as well as the amount of info we've collected
        self.tick_runmean = self.tick_open #need a running mean
        self.levtick_runmean = self.levtick_open
        self.tick_numticks = 1 #counts number of total ticks
        self.levtick_numticks = 1
        self.tick_runsum = self.tick_open
        self.levtick_runsum = self.levtick_open
        self.tick_openpercent = 1
        self.levtick_openpercent = 1
        self.blagh = 1 #tracks divergence between the two instruments
        self.order_blagh = 1

        # How many seconds we will wait in between updating the streak values
        self.tick_size = 1

        # The percentage of our buying power that we will allocate to a new
        # position after a streak reset
        self.base_bet = 10

        # These variables track the information about the current streak
        self.streak_count = 0
        self.streak_start = 0
        self.streak_increasing = True

        # When this variable is not None, we have an order open
        self.current_order = None

        # The closing price of the last aggregate we saw
        self.last_price = 0

        # used to use tick data as second aggs data (mot every tick, but sec)
        self.last_trade_time = datetime.datetime.utcnow()

        # The connection to the Alpaca API
        self.api = tradeapi.REST(
            self.key_id,
            self.secret_key,
            self.base_url
        )

        # Cancel any open orders so they don't interfere with this script
        self.api.cancel_all_orders()

        try:
            self.position = int(self.api.get_position(self.symbol).qty)
        except:
            # No position exists
            self.position = 0

        # Figure out how much money we have to work with, accounting for margin
        account_info = self.api.get_account()
        print("account infor", account_info)
        self.equity = abs(float(account_info.long_market_value)) + abs(float(account_info.short_market_value))
        self.margin_multiplier = float(account_info.multiplier)
        total_buying_power = self.margin_multiplier * self.equity
        print("current invested", self.equity )

        print("liquidating just in case")
        self.api.close_all_positions()
        print("new equity")
        self.equity = abs(float(account_info.long_market_value)) + abs(float(account_info.short_market_value))
        print(self.equity)


    def send_order(self, sym_quant, lev_sym_quant):
          print("order sent for", sym_quant, self.tick_symbol, lev_sym_quant, self.levtick_symbol)
          # We don't want to have two orders open at once
          self.api.cancel_all_orders()
          print("canceled open orders")

          #try order for unleveraged ticker
          try:
              print("made order for", self.tick_symbol)
              if(sym_quant > 0):
                self.api.submit_order(
                            self.tick_symbol, 
                            int(sym_quant), 
                            'buy',
                            'limit', 
                            'fok', 
                            round(self.last_tick_val, 2)
                        )
              else:
                self.api.submit_order(
                            self.tick_symbol, 
                            int(-1*sym_quant), 
                            'sell',
                            'limit', 
                            'fok', 
                            round(self.last_tick_val, 2)
                        )

          except Exception as e:
              print(e)
              print("trade error, liquidating")
              self.api.close_all_positions()

          #try order for leveraged ticker
          try:

            print("made order for", self.levtick_symbol)
            if(lev_sym_quant > 0):

              self.api.submit_order(
                          self.levtick_symbol, 
                          int(lev_sym_quant), 
                          'buy',
                          'limit', 
                          'fok', 
                          round(self.last_levtick_val, 2)
                      )
            else:

              self.api.submit_order(
                          self.levtick_symbol, 
                          int(-1*lev_sym_quant), 
                          'sell',
                          'limit', 
                          'fok', 
                          round(self.last_levtick_val, 2)
                      )
          except Exception as e:
              print(e)
              print("trade error, liquidating")
              self.api.close_all_positions()
          longval = abs(float(self.api.get_account().long_market_value))
          shortval = abs(float(self.api.get_account().short_market_value))
          #ensure both arms of the trade went through
          if(longval == 0 or shortval == 0):
              print("only one arm executed, liquidating", longval, shortval)
              self.api.close_all_positions()

          self.equity = abs(float(self.api.get_account().long_market_value)) + abs(float(self.api.get_account().short_market_value))

    def process_current_tick(self, blagh):
          # Update  info
          print("entered tick processing logic with blagh", blagh)
          if(self.equity > 0):
            #check if we need to sell anything and what current value is

            print("entered sell check processing")
            print("current long", self.api.get_account().long_market_value, "current short", self.api.get_account().short_market_value)
            self.equity = abs(float(self.api.get_account().long_market_value)) + abs(float(self.api.get_account().short_market_value))
            if((self.order_blagh > 0 and blagh < 0) or (self.order_blagh < 0 and blagh > 0)):
              print("closed out a trade")
              print("trade value", self.equity)
              self.api.close_all_positions()
              self.equity = abs(float(self.api.get_account().long_market_value)) + abs(float(self.api.get_account().short_market_value))
              print("closed out to value", self.equity)
          if self.equity == 0:

            #buy depending on divergence
            print("entered buy check processing")
            if(self.blagh > 0 and self.tick_numticks > 1 and self.levtick_numticks > 1):
              lev_sym_quant = 0
              sym_quant = 3
              lev_price = self.last_levtick_val
              sym_price = self.last_tick_val

              lev_sym_quant = sym_quant*sym_price/3
              lev_sym_quant = lev_sym_quant/lev_price
              lev_sym_quant = -1*int(lev_sym_quant)

              self.order_blagh = blagh

              print("blagh", blagh, "sent order quantity", sym_quant, lev_sym_quant, "with value", sym_quant*sym_price + lev_sym_quant*lev_price)

              self.send_order(sym_quant, lev_sym_quant)

            elif(self.blagh < 0):

              #sell depending on divergence
              lev_sym_quant = 0
              sym_quant = -3
              lev_price = self.last_levtick_val
              sym_price = self.last_tick_val

              lev_sym_quant = sym_quant*sym_price/3
              lev_sym_quant = lev_sym_quant/lev_price
              lev_sym_quant = -1*int(lev_sym_quant)

              self.order_blagh = blagh

              print("blagh", blagh, "sent order quantity", sym_quant, lev_sym_quant, "with value", sym_quant*sym_price + lev_sym_quant*lev_price)

              self.send_order(sym_quant, lev_sym_quant)


    def start_trading(self):

        # Listen for second aggregates and perform trading logic

           
        async def handle_tick_bar(bar):
            print("bar", bar)
            self.tick_numticks +=1
            print("num ticks", self.tick_numticks)

            if(self.tick_numticks) % self.tick_size == 0:
                # It's time to update

                self.last_tick_val = bar.close #keep track of the last price of the instrument

                self.tick_runsum += bar.close #update runsum
                self.tick_runmean = self.tick_runsum / self.tick_numticks #update runmean
                self.tick_openpercent = 3*( (bar.close / self.tick_open)-1) + 1 #keep track of the "theoretical" 3x leveraged returns
                self.blagh = self.levtick_openpercent / self.tick_openpercent #calculate divergence
                print("grabbed new bar for ", self.tick_symbol)
                print("bar val", self.last_tick_val, "runmean", self.tick_runmean, "percent change from open", self.tick_openpercent, "blagh", self.blagh)
                self.process_current_tick(self.blagh) #now we want to check if we have no open positions and act accordingly

        async def handle_levtick_bar(bar):
            print("bar", bar)
            self.levtick_numticks +=1
            print("num levticks", self.levtick_numticks)

            if(self.levtick_numticks) % self.tick_size == 0:
                # It's time to update

                self.last_levtick_val = bar.close #keep track of the last price of the instrument

                self.levtick_runsum += bar.close #update runsum
                self.levtick_runmean = self.levtick_runsum / self.levtick_numticks #update runmean
                self.levtick_openpercent = (bar.close / self.levtick_open) #keep track of the "theoretical" 3x leveraged returns
                self.blagh = self.levtick_openpercent / self.tick_openpercent #calculate divergence
                print("grabbed new bar for ", self.levtick_symbol)
                print("bar val", self.last_levtick_val, "runmean", self.levtick_runmean, "percent change from open", self.levtick_openpercent, "blagh", self.blagh)
                self.process_current_tick(self.blagh) #now we want to check if we have no open positions and act accordingly
        

        
        

        




        conn = Stream(
        self.key_id,
        self.secret_key,
        base_url=self.base_url,
        data_feed='iex')  # <- replace to SIP if you have PRO subscription

        print("connection created")

        conn.subscribe_bars(handle_tick_bar, self.tick_symbol)
        conn.subscribe_bars(handle_levtick_bar, self.levtick_symbol)
        print("bars subscribed")

        conn.run()

    


        

In [ ]:
trader = LEFTTRADER()
print("instantiated trader")



account infor Account({   'account_blocked': False,
    'account_number': 'PA3NKZ9F2DLO',
    'accrued_fees': '0',
    'buying_power': '199207.34',
    'cash': '99603.67',
    'created_at': '2021-10-04T18:21:29.569662Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 37,
    'daytrading_buying_power': '0',
    'equity': '99603.67',
    'id': '8b3bd92d-bf65-4471-866a-33a49387aad0',
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99603.67',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '99603.67',
    'regt_buying_power': '199207.34',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '100000',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})
current invested 0.

In [ ]:
trader.__dict__

{'api': <alpaca_trade_api.rest.REST at 0x7f7c84c125d0>,
 'base_bet': 10,
 'base_url': 'https://paper-api.alpaca.markets',
 'blagh': 1,
 'current_order': None,
 'data_url': 'https://paper-api.alpaca.markets',
 'equity': 0.0,
 'key_id': 'PKZBF19BMWCUWI0SA9E3',
 'last_levtick_val': 0,
 'last_price': 0,
 'last_tick_val': 0,
 'last_trade_time': datetime.datetime(2022, 6, 17, 19, 57, 3, 848871),
 'levtick_numticks': 1,
 'levtick_open': 32.0,
 'levtick_openpercent': 1,
 'levtick_runmean': 32.0,
 'levtick_runsum': 32.0,
 'levtick_symbol': 'UPRO',
 'margin_multiplier': 2.0,
 'order_blagh': 1,
 'position': 0,
 'secret_key': 'uX2AdG5eqYzu80wf5PsjbrbvFoKR9NOx2iuDQHfP',
 'streak_count': 0,
 'streak_increasing': True,
 'streak_start': 0,
 'tick_numticks': 1,
 'tick_open': 365.51,
 'tick_openpercent': 1,
 'tick_runmean': 365.51,
 'tick_runsum': 365.51,
 'tick_size': 1,
 'tick_symbol': 'SPY'}

In [ ]:
trader.start_trading()


connection created
bars subscribed
bar Bar({   'close': 366.325,
    'high': 366.8,
    'low': 366.325,
    'open': 366.8,
    'symbol': 'SPY',
    'timestamp': 1655495820000000000,
    'trade_count': 320,
    'volume': 52936,
    'vwap': 366.498471})
num ticks 2
grabbed new bar for  SPY
bar val 366.325 runmean 365.9175 percent change from open 1.0066892834669363 blagh 0.9933551657131989
entered tick processing logic with blagh 0.9933551657131989
entered buy check processing
bar Bar({   'close': 366.11,
    'high': 366.44,
    'low': 366.11,
    'open': 366.33,
    'symbol': 'SPY',
    'timestamp': 1655495880000000000,
    'trade_count': 393,
    'volume': 76650,
    'vwap': 366.221671})
num ticks 3
grabbed new bar for  SPY
bar val 366.11 runmean 365.98166666666674 percent change from open 1.0049246258652298 blagh 0.9950995072282265
entered tick processing logic with blagh 0.9950995072282265
entered buy check processing
bar Bar({   'close': 365.81,
    'high': 366.2,
    'low': 365.78,

error during websocket communication: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/nest_asyncio.py", line 35, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.7/dist-packages/nest_asyncio.py", line 83, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.7/dist-packages/nest_asyncio.py", line 106, in _run_once
    event_list = self._selector.select(timeout)
  File "/usr/lib/python3.7/selectors.py", line 468, in select
    fd_event_list = self._selector.poll(timeout, max_ev)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/alpaca_trade_api/stream.py", line 254, in _run_forever
    await self._consume()
  File "/usr/local/lib/python3.7/dist-packages/alpaca_trade_api/stream.py", line 127, in _consume
    r = await asyncio.wait_for(self._ws.recv(), 5)
  File "/usr/lib/python3.7/async

keyboard interrupt, bye
